In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from perplexity import Perplexity
import json

# Load a SLM
model_name = "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
dataset = load_dataset("JeanKaddour/minipile", split='validation')

NameError: name 'load_dataset' is not defined

In [3]:
import pandas as pd
dataset_path = "../../eve-training/data/ESA_EO_webscrape.jsonl"


# Load the data from jsonl
df = pd.read_json(dataset_path, lines=True)
len(df.loc[0]['text'])

158042

In [24]:
perp = Perplexity(df.head(4), model, tokenizer, batch_size=2)

In [25]:
result = perp()  

In [23]:
result

35.287845611572266

In [14]:
%load_ext autoreload
%autoreload 2

In [6]:
import evaluate
perplexity = evaluate.load("perplexity", module_type="metric")
input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
results = perplexity.compute(model_id='gpt2',
                             add_start_token=False,
                             predictions=input_texts)
print(results)

/home/elements72/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

{'perplexities': [32.25418472290039, 1499.6954345703125, 408.27459716796875], 'mean_perplexity': 646.7414054870605}


In [9]:
import datasets

perplexity = evaluate.load("perplexity", module_type="metric")
input_texts = datasets.load_dataset("wikitext",
                                    "wikitext-2-raw-v1",
                                    split="test")["text"][10:15]
input_texts = [s for s in input_texts if s != '']
len(input_texts)

3

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import datasets
import evaluate

from perplexity import compute
from tqdm import tqdm
import torch

In [26]:
%load_ext autoreload
%autoreload 2

In [1]:
def compute_axo(references, model_name, stride=512, max_seq_len=512, batch_size=4): 
        """
        Compute perplexity in a fixed length sliding window across the sequence.
        """
        assert references is not None, "Missing parameter: references"
        model = AutoModelForCausalLM.from_pretrained(model_name)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        tokenizer.pad_token = tokenizer.eos_token

        model.eval()

        references_tokenized = tokenizer(
            references,
            add_special_tokens=False,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt",
            return_attention_mask=True,
        )
        input_ids: Tensor = references_tokenized["input_ids"]  # type: ignore
        input_ids = input_ids.to(model.device)

        sequence_length = input_ids.size(1)
        print(input_ids)

        losses = []
        prev_end_loc = 0
        for begin_loc in tqdm(
            range(0, sequence_length, stride), desc="Sliding window"):
            end_loc = min(begin_loc + max_seq_len, sequence_length)
            trg_len = end_loc - prev_end_loc
            input_ids_slice = input_ids[:, begin_loc:end_loc]
            labels_slice = input_ids_slice.clone()
            labels_slice[:, :-trg_len] = -100

            with torch.no_grad():
                outputs = model(
                    input_ids=input_ids_slice, labels=labels_slice
                )

            losses.append(outputs.loss)

            prev_end_loc = end_loc
            if end_loc == sequence_length:
                break

        perplexity = torch.exp(torch.stack(losses).mean()).item()

        return {
            "score": perplexity,
        }

In [22]:
texts = ['lorem ipsum', 'Happy Birthday!', 'Bienvenue']

batch_size = 4

ppl_evaluate = evaluate.load("perplexity", module_type="metric")
results_evaluate = ppl_evaluate.compute(model_id='gpt2',
                               add_start_token=False,
                               predictions=texts, batch_size=4)

results_evaluate

  0%|          | 0/1 [00:00<?, ?it/s]

{'perplexities': [32.25418472290039, 1499.6954345703125, 408.27459716796875],
 'mean_perplexity': 646.7414054870605}

In [29]:
compute_axo(texts, 'gpt2')

tensor([[   75, 29625,   220,  2419,   388],
        [25082, 33511,     0, 50256, 50256],
        [   33,  2013,  4080, 50256, 50256]])


Sliding window:   0%|          | 0/1 [00:00<?, ?it/s]


{'score': 534.8441162109375}

In [31]:
results_window = compute(texts, 'gpt2', batch_size=4)

results_window

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[   75, 29625,   220,  2419,   388],
        [25082, 33511,     0, 50256, 50256],
        [   33,  2013,  4080, 50256, 50256]])



100%|██████████| 1/1 [00:00<00:00,  4.50it/s]


{'mean_perplexity': 178.28263346354166}

In [ ]:
from datasets import Dataset

data = {
    "question": [
        "Who developed the theory of relativity?",
        "What is the capital of France?",
        "Which planets are known as gas giants?"
    ],
    "choices": [
        ["A) Isaac Newton", "B) Albert Einstein", "C) Galileo Galilei", "D) Nikola Tesla"],
        ["A) Berlin", "B) Madrid", "C) Paris", "D) Rome"],
        ["A) Mercury", "B) Venus", "C) Jupiter", "D) Saturn", "E) Uranus", "F) Neptune"]
    ],
    "answer": ["B", "C", "C"]  # Correct answer keys
}

dataset = Dataset.from_dict(data)
print(dataset)


In [1]:
import datasets


data = datasets.load_dataset("eve-esa/eve-is-mcqa")

/Users/antoniolopez/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['question', 'choices', 'answers'],
        num_rows: 432
    })
})